In [30]:
import sys
print("Jupyter Notebook이 현재 사용 중인 파이썬의 전체 경로입니다:")
print(sys.executable)

Jupyter Notebook이 현재 사용 중인 파이썬의 전체 경로입니다:
f:\dev\Competition\k_ets\K-ETS_Dashboard\.venv\Scripts\python.exe


In [31]:
import os
import time
import fitz  # PyMuPDF
import requests
import json
from glob import glob
from dotenv import load_dotenv
from pathlib import Path

from langchain_core.documents import Document
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_upstage import UpstageEmbeddings, ChatUpstage
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone, ServerlessSpec

from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain import hub

# --- 1. 환경 설정 및 상수 정의 ---
load_dotenv()

UPSTAGE_API_KEY = os.getenv("UPSTAGE_API_KEY")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
if not UPSTAGE_API_KEY or not PINECONE_API_KEY:
    raise ValueError("API keys for Upstage and Pinecone must be set in the .env file.")

# 프로젝트 루트 자동 감지
def find_project_root():
    current = Path.cwd()
    markers = ["requirements.txt", "README.md", ".env", "chatbot_app.py"]
    
    for directory in [current] + list(current.parents):
        if any((directory / marker).exists() for marker in markers):
            return directory
        if directory.name == "K-ETS_Dashboard":
            return directory
    return current

# 경로 설정
PROJECT_ROOT = find_project_root()
DOCS_FOLDER = PROJECT_ROOT / "docs"

print(f"Project root: {PROJECT_ROOT}")
print(f"Docs folder: {DOCS_FOLDER}")

INDEX_NAME = "carbon-rag"
CHUNK_SIZE = 1500
CHUNK_OVERLAP = 200

Project root: f:\dev\Competition\k_ets\K-ETS_Dashboard
Docs folder: f:\dev\Competition\k_ets\K-ETS_Dashboard\docs


In [32]:
# 파일 찾기
def get_pdf_files(docs_folder: str) -> list:
    """docs 폴더에서 모든 PDF 파일을 찾아 반환"""
    pdf_pattern = os.path.join(docs_folder, "*.pdf")
    pdf_files = glob(pdf_pattern)
    
    if not pdf_files:
        raise ValueError(f"No PDF files found in {docs_folder}")
    
    print(f"Found {len(pdf_files)} PDF files:")
    for i, pdf_file in enumerate(pdf_files, 1):
        filename = os.path.basename(pdf_file)
        file_size = os.path.getsize(pdf_file) / (1024 * 1024)  # MB
        print(f"  {i}. {filename} ({file_size:.1f} MB)")
    
    return pdf_files



In [33]:
# --- 2. Upstage Document Parse API를 이용한 PDF 처리 ---
def parse_multiple_pdfs_with_upstage(pdf_files: list, batch_size: int = 10) -> str:
    """
    여러 PDF 파일을 Upstage Document Parse API로 처리하고 통합된 HTML 내용을 반환합니다.
    """
    print(f"--- Starting Multiple PDF Processing for {len(pdf_files)} files ---")
    
    all_html_content = ""
    
    for file_index, input_file in enumerate(pdf_files, 1):
        filename = os.path.basename(input_file)
        print(f"\nProcessing file {file_index}/{len(pdf_files)}: {filename}")
        
        try:
            # 개별 PDF 처리
            html_content = parse_pdf_with_upstage(input_file, batch_size)
            
            if html_content:
                # 파일 구분자 추가
                separator = f"\n\n{'='*80}\n파일: {filename}\n{'='*80}\n\n"
                all_html_content += separator + html_content
                print(f"  처리 성공!!! : {len(html_content):,} characters")
            else:
                print(f"  실패: {filename}")
                
        except Exception as e:
            print(f"  Error processing {filename}: {e}")
            continue
    
    print(f"\n--- 다중 PDF 처리 완료 ---")
    print(f"Total content length: {len(all_html_content):,} characters")
    return all_html_content


def parse_pdf_with_upstage(input_file: str, batch_size: int = 10) -> str:
    """
    Upstage Document Parse API를 사용하여 PDF를 처리하고, 전체 HTML 내용을 반환합니다.
    - PDF를 작은 페이지 묶음으로 분할
    - 각 묶음을 API로 전송하여 JSON(HTML 포함) 결과 받기
    - 모든 HTML 결과를 하나로 통합
    """
    print(f"--- Starting PDF Processing for: {input_file} ---")

    # Step 2.1: PDF를 작은 파일로 분할
    print(f"\n[Step 1/3] Splitting PDF into chunks of {batch_size} pages...")
    input_pdf = fitz.open(input_file)
    num_pages = len(input_pdf)
    
    split_files = []
    for start_page in range(0, num_pages, batch_size):
        end_page = min(start_page + batch_size, num_pages) - 1
        input_file_basename = os.path.splitext(input_file)[0]
        output_file = f"{input_file_basename}_{start_page}_{end_page}.pdf"
        
        with fitz.open() as output_pdf:
            output_pdf.insert_pdf(input_pdf, from_page=start_page, to_page=end_page)
            output_pdf.save(output_file)
        split_files.append(output_file)
        print(f"  - 분할된 파일: {output_file}")
    input_pdf.close()

    # Step 2.2: 분할된 각 PDF에 대해 Upstage Parse API 호출
    print("\n[Step 2/3] Calling Upstage Document Parse API for each split file...")
    json_files = []
    for short_input_file in split_files:
        short_output_file = os.path.splitext(short_input_file)[0] + ".json"
        try:
            print(f"  - Processing {short_input_file}...")
            response = requests.post(
                "https://api.upstage.ai/v1/document-digitization",
                headers={"Authorization": f"Bearer {UPSTAGE_API_KEY}"},
                data={"base64_encoding": "['figure']", "model": "document-parse"},
                files={"document": open(short_input_file, "rb")},
            )
            response.raise_for_status()  # 200이 아닌 경우 예외 발생
            with open(short_output_file, "w", encoding="utf-8") as f:
                json.dump(response.json(), f, ensure_ascii=False, indent=4)
            json_files.append(short_output_file)
            print(f"  - Saved API response to {short_output_file}")
        except requests.exceptions.RequestException as e:
            print(f"  - 에러 {short_input_file}: {e}")
            # 이미지 파일 추출 등은 여기서 생략하고, 텍스트 추출에 집중합니다.

    # Step 2.3: JSON 파일에서 콘텐츠 통합 (단일 로직)
    print("\n[Step 3/3] Consolidating content from all JSON files...")
    full_html_content = ""
    
    for json_file in json_files:
        with open(json_file, "r", encoding="utf-8") as f:
            data = json.load(f)
            
            # Upstage Document Parse API 응답 구조에 맞게 처리
            if "content" in data and data["content"]:
                if isinstance(data["content"], dict):
                    # content가 dict 타입인 경우 적절히 처리
                    if 'html' in data["content"]:
                        content_text = data["content"]["html"]
                    elif 'text' in data["content"]:
                        content_text = data["content"]["text"]
                    else:
                        # dict 전체를 문자열로 변환
                        content_text = str(data["content"])
                else:
                    content_text = str(data["content"])
                
                full_html_content += content_text
                print(f"  - 성공!!! 콘텐츠 추출: {len(content_text)} 문자")
            else:
                print(f"  - 실패!!! 'content' 키를 찾을 수 없음: {json_file}")
    
    # 임시 파일 정리
    for f in split_files + json_files:
        try:
            os.remove(f)
        except:
            pass  # 파일 삭제 실패해도 무시

    print(f"--- 단일 PDF 처리 완료 - Total: {len(full_html_content)} 문자 ---")
    return full_html_content

In [34]:
# --- 3. LangChain을 이용한 RAG 파이프라인 구축 ---

# Step 3.1: 파싱된 HTML을 LangChain Document로 변환 및 분할
def get_document_splits(html_content: str): # source_files: list
    """전처리된 HTML 콘텐츠를 LangChain Document로 변환하고 텍스트 분할을 수행합니다."""
    print("\n--- Starting LangChain Document Processing ---")
    
    # 전처리된 HTML 콘텐츠로 Document 객체 생성
    doc = Document(
        page_content=html_content, 
        metadata={
            "source": "upstage_parsed_content",  # 전처리된 콘텐츠임을 명시
            # "original_file": [os.path.basename(f) for f in source_files],  # 여러 파일 정보
            # "processing_method": "upstage_document_parse_api",
            "content_length": len(html_content)
        }
    )

    # 텍스트 분할기 설정 (HTML 구조 고려)
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=CHUNK_SIZE,
        chunk_overlap=CHUNK_OVERLAP,
        separators=["\n\n", "\n", " ", ""]  # HTML 구조에 맞는 구분자
    )
    
    # 수이미 준비된 Document 객체를 직접 분할
    document_list = text_splitter.split_documents([doc])
    
    print(f"  - 성공!!! Document split into {len(document_list)} chunks")
    print(f"  - 성공!!! Total content length: {len(html_content):,} characters")
    print(f"  - 성공!!! Average chunk size: {len(html_content) // len(document_list):,} characters")
    
    return document_list

# Step 3.2: 임베딩 및 벡터 저장소 설정
def setup_vector_store(document_list):
    """문서 청크를 임베딩하고 Pinecone 벡터 저장소에 저장합니다."""
    print(f"\n--- Setting up Vector Store with {len(document_list)} documents ---")
    
    # Upstage 임베딩 모델 초기화
    embeddings = UpstageEmbeddings(model="embedding-query")
    
    # Pinecone 초기화
    pinecone_api_key = os.getenv("PINECONE_API_KEY")
    pc = Pinecone(api_key=pinecone_api_key)
    
    # 실제 문서와 함께 벡터 저장소 초기화
    try:
        # 기존 인덱스가 있는지 확인하고 문서 추가
        database = PineconeVectorStore(
            embedding=embeddings,
            index_name=INDEX_NAME
        )
        
        # 문서를 배치로 업로드
        batch_size = 50  # 배치 크기를 줄여서 안정성 향상
        total_batches = (len(document_list) + batch_size - 1) // batch_size
        
        print(f"  - Uploading {len(document_list)} documents in {total_batches} batches...")
        
        for i in range(0, len(document_list), batch_size):
            batch = document_list[i:i + batch_size]
            batch_num = (i // batch_size) + 1
            
            print(f"    - Batch {batch_num}/{total_batches}: {len(batch)} documents")
            database.add_documents(batch)
            
            # API 제한을 고려, 딜레이 1초
            time.sleep(1)
        
        print("  - 성공!!! 벡터스토어 설정 끝")
        return database
        
    except Exception as e:
        print(f"  - 벡터스토어 설정 실패: {e}")
        print("  - 새 벡터스토어 만드는 중...")
        
        # 실패시 새로운 벡터 저장소 생성
        database = PineconeVectorStore.from_documents(
            documents=document_list,
            embedding=embeddings,
            index_name=INDEX_NAME
        )
        print("  - 성공!!! 새 벡터스토어 생성")
        return database

# Step 3.3: LCEL을 사용한 RAG 체인 구성
def create_rag_chain(vectorstore):
    """LCEL을 사용하여 RAG 체인을 생성"""
    print("\n--- Creating RAG Chain using LCEL ---")

    # 1. Retriever (벡터 저장소에서 관련 문서를 검색)
    retriever = vectorstore.as_retriever(
        search_kwargs={'k': 4}  # 상위 4개 관련 문서 검색
    )
    
    # 2. LLM (Upstage Solar 모델)
    llm = ChatUpstage(
        model="solar-mini", 
        temperature=0  # 일관된 답변을 위해 temperature 0
    )
    
    # 3. Prompt (Hub에서 RAG 프롬프트 가져오기)
    try:
        prompt = hub.pull("rlm/rag-prompt")
        print("  - Loaded RAG prompt from LangChain Hub")
    except Exception as e:
        print(f"  - Failed to load prompt from hub: {e}")
        print("  - Using custom prompt...")
        
        # 대안: 커스텀 프롬프트 생성
        from langchain_core.prompts import ChatPromptTemplate
        
        prompt = ChatPromptTemplate.from_template("""
당신은 문서 분석 전문가입니다. 주어진 맥락을 바탕으로 질문에 정확하고 자세히 답변해주세요.

맥락:
{context}

질문: {question}

답변:
""")

    # 문서를 하나의 문자열로 포맷하는 함수
    def format_docs(docs):
        formatted = "\n\n".join([
            f"[문서 {i+1}]\n{doc.page_content}" 
            for i, doc in enumerate(docs)
        ])
        return formatted

    # 4. LCEL Chain 구성
    rag_chain = (
        {"context": retriever | format_docs, "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
    )
    
    print("  - RAG Chain Created Successfully")
    print(f"  - Retriever will search top {retriever.search_kwargs['k']} similar documents")
    
    return rag_chain

In [35]:
# --- 4. 메인 실행 로직 ---
if __name__ == "__main__":
    # 1. docs 폴더에서 모든 PDF 파일 찾기
    pdf_files = get_pdf_files(DOCS_FOLDER)
    
    # 2. 모든 PDF 파일을 Upstage API로 처리하여 통합 HTML 생성
    html_content = parse_multiple_pdfs_with_upstage(pdf_files)
    if not html_content:
        raise ValueError("Failed to parse PDF files or extract HTML content.")
    
    # 3. HTML을 LangChain 문서로 변환 및 분할
    document_splits = get_document_splits(html_content)

    # 4. 벡터 저장소 설정 및 문서 저장
    vector_store = setup_vector_store(document_splits)
    
    # 5. RAG 체인 생성
    qa_chain = create_rag_chain(vector_store)

    # 6. 질문하고 답변받기
    print("\n--- Starting Q&A ---")
    query = '온실가스 배출량 산정을 위한 자료를 제출하지 아니하거나 거짓으로 제출한 경우 과태료는?'
    print(f"\n[Question]: {query}")
    
    ai_message = qa_chain.invoke(query)
    
    print("\n[Answer]:")
    print(ai_message)

Found 9 PDF files:
  1. 2024 배출권거래제 운영결과보고서(국문).pdf (21.8 MB)
  2. 2050 탄소중립녹색성장위원회 운영세칙(국무조정실훈령)(제218호)(20250224).pdf (0.1 MB)
  3. 기후위기 대응을 위한 탄소중립ㆍ녹색성장 기본법 시행령(대통령령)(제35435호)(20250408).pdf (0.2 MB)
  4. 기후위기 대응을 위한 탄소중립ㆍ녹색성장 기본법(법률)(제20514호)(20241022).pdf (0.2 MB)
  5. 기후위기 대응을 위한 탄소중립ㆍ녹색성장 기본법(법률)(제20514호)(20241022)_0_9.pdf (0.2 MB)
  6. 기후위기 대응을 위한 탄소중립ㆍ녹색성장 기본법(법률)(제20514호)(20241022)_10_19.pdf (0.1 MB)
  7. 기후위기 대응을 위한 탄소중립ㆍ녹색성장 기본법(법률)(제20514호)(20241022)_20_25.pdf (0.1 MB)
  8. 유엔기후변화협약 및 파리협정에 따른 제1차 격년투명성보고서(BTR) 및 제5차 국가보고서(NC).pdf (18.6 MB)
  9. 제4차 배출권거래제 기본계획(2024.12.).pdf (1.2 MB)
--- Starting Multiple PDF Processing for 9 files ---

Processing file 1/9: 2024 배출권거래제 운영결과보고서(국문).pdf
--- Starting PDF Processing for: f:\dev\Competition\k_ets\K-ETS_Dashboard\docs\2024 배출권거래제 운영결과보고서(국문).pdf ---

[Step 1/3] Splitting PDF into chunks of 10 pages...
  - 분할된 파일: f:\dev\Competition\k_ets\K-ETS_Dashboard\docs\2024 배출권거래제 운영결과보고서(국문)_0_9.pdf
  - 분할된 파일: f:\dev\Competition\k_ets\K-ET

f:\dev\Competition\k_ets\K-ETS_Dashboard\.venv\Lib\site-packages\langsmith\client.py:277: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


  - Loaded RAG prompt from LangChain Hub
  - RAG Chain Created Successfully
  - Retriever will search top 4 similar documents

--- Starting Q&A ---

[Question]: 온실가스 배출량 산정을 위한 자료를 제출하지 아니하거나 거짓으로 제출한 경우 과태료는?

[Answer]:
온실가스 배출량 산정을 위한 자료를 제출하지 아니하거나 거짓으로 제출한 경우 1천만원 이하의 과태료를 부과합니다.


In [37]:
print("\n--- Starting Q&A ---")
query = '격년투명성보고서에서 선정한 주요 지표는?'
print(f"\n[Question]: {query}")

ai_message = qa_chain.invoke(query)

print("\n[Answer]:")
print(ai_message)


--- Starting Q&A ---

[Question]: 격년투명성보고서에서 선정한 주요 지표는?

[Answer]:
격년투명성보고서에서 선정한 주요 지표는 '연간 온실가스 총배출량(LULUCF 제외)'입니다. 이 지표는 에너지, 산업공정, 농업 등에서 발생하는 온실가스 배출량을 합산한 값으로, 토지이용·토지이용 변화 및 임업(Land Use, Land-Use Change and Forestry, 이하 LULUCF) 분야는 제외됩니다.


In [38]:
print("\n--- Starting Q&A ---")
query = '연구개발 투자 현황을 알려줘'
print(f"\n[Question]: {query}")

ai_message = qa_chain.invoke(query)

print("\n[Answer]:")
print(ai_message)


--- Starting Q&A ---

[Question]: 연구개발 투자 현황을 알려줘

[Answer]:
기후기술 국가연구개발사업에 투자한 규모는 2022년 기준으로 총 3조 9,073억 원입니다. 이는 국가 전체 연구개발비의 약 12.9%를 차지합니다. 온실가스 감축 부문에는 2조 9,072억 원(74.4%), 적응 부문에는 1조 원(25.6%)을 투자하였습니다. 온실가스 감축 부문 중에서 가장 높은 비중을 차지하는 분야는 에너지 생산으로, 1조 611억 원을 이 분야에 투자하였습니다. 그 다음으로 에너지 효율 분야에 9,761억 원, 연/원료 대체 분야에 4,666억 원을 투자하였습니다. 기후변화에 대한 적응 부문 중에서 가장 높은 투자 비중을 차지하는 분야는 피해관리 및 탄력성 제고 분야로, 7,779억 원을 이 분야에 투자하였습니다. 그다음으로 기후변화 모니터링 분야에 1,913억 원, 기후영향평가 및 진단 분야에 196억 원을 투자하였습니다.
